In [ ]:
import os
import cv2
import sys
import open3d
import torch
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

sys.path.append('../')
from unidepth.utils import colorize


In [ ]:
device = 'cuda:0'
version = "v1"
backbone = "ViTL14"
frame_id = 0
root = '/home/user/UniDepth' # TODO: change this to your own path


In [ ]:
# camera details
info_path = os.path.join(root, 'assets/3RScan/5630cfcf-12bf-2860-8784-83d28a611a83', 'sequence', '_info.txt')
with open(info_path) as fp:
    for line in fp:
        sline = line.strip()
        if (sline.rfind("m_colorWidth", 0) == 0):
            # color width
            color_width = int(sline[sline.find("= ") + 2:])
        elif (sline.rfind("m_colorHeight", 0) == 0):
            # color height
            color_height = int(sline[sline.find("= ") + 2:])
        elif (sline.rfind("m_depthWidth", 0) == 0):
            # depth width
            depth_width = int(sline[sline.find("= ") + 2:])
        elif (sline.rfind("m_depthHeight", 0) == 0):
            # color height
            depth_height = int(sline[sline.find("= ") + 2:])
        elif (sline.rfind("m_depthShift", 0) == 0):
            # depth shift
            # The depth map is a 16 bit image and stores depth values as an unsigned short in millimeter.
            integer_depth_scale = 1 / float(sline[sline.find("= ") + 2:])
        elif (sline.rfind("m_calibrationColorIntrinsic", 0) == 0):
            # color intrinsics
            color_intrinsics = np.identity(3, dtype=np.float32)
            model = sline[sline.find("= ") + 2:].split(' ')
            color_intrinsics[0, 0] = float(model[0])
            color_intrinsics[1, 1] = float(model[5])
            color_intrinsics[0, 2] = float(model[2])
            color_intrinsics[1, 2] = float(model[6])
        elif (sline.rfind("m_calibrationDepthIntrinsic", 0) == 0):
            # depth intrinsics
            depth_intrinsics = np.identity(3, dtype=np.float32)
            model = sline[sline.find("= ") + 2:].split(' ')
            depth_intrinsics[0, 0] = float(model[0])
            depth_intrinsics[1, 1] = float(model[5])
            depth_intrinsics[0, 2] = float(model[2])
            depth_intrinsics[1, 2] = float(model[6])
        else:
            pass

# camera coordinates to world coordinates transformation; pose = [R | t]
pose_path = os.path.join(root, 'assets/3RScan/5630cfcf-12bf-2860-8784-83d28a611a83', 'sequence', f'frame-{frame_id:06}.pose.txt')
pose = np.loadtxt(pose_path)
R = pose[:3, :3] # rotation
t = pose[:3, 3] # translation

# world coordinates to camera coordinates transformation; inv_pose = [inv_R | -inv_R * t]
inv_R = R.T
inv_t = -R.T @ t

# 4x4 transformation
T = np.column_stack((inv_R, inv_t))
T = np.vstack((T, (0, 0, 0, 1)))

#
width = color_width
height = color_height
intrinsics_gt = color_intrinsics

# color image
rgb_path = os.path.join(root, 'assets/3RScan/5630cfcf-12bf-2860-8784-83d28a611a83', 'sequence', f'frame-{frame_id:06}.color.jpg')
rgb = cv2.imread(rgb_path, cv2.IMREAD_UNCHANGED)
rgb = cv2.resize(rgb, dsize=(width, height), interpolation=cv2.INTER_LINEAR) # HACK: resize color image to match color intrinsics
rgb = cv2.cvtColor(rgb, cv2.COLOR_BGR2RGB)

# depth image
depth_path = os.path.join(root, 'assets/3RScan/5630cfcf-12bf-2860-8784-83d28a611a83', 'sequence', f'frame-{frame_id:06}.depth.pgm')
depth_gt = cv2.imread(depth_path, cv2.IMREAD_UNCHANGED)
depth_gt = cv2.resize(depth_gt, dsize=(width, height), interpolation=cv2.INTER_NEAREST) # HACK: resize depth image to match color intrinsics
depth_gt = depth_gt.astype(np.float32) * integer_depth_scale


In [ ]:
# Open3D rgbd image
o3d_color = open3d.geometry.Image(rgb)
o3d_depth = open3d.geometry.Image(depth_gt)
o3d_rgbd = open3d.geometry.RGBDImage.create_from_color_and_depth(
    color=o3d_color,
    depth=o3d_depth,
    depth_scale=1.0,
    depth_trunc=10.0,
    convert_rgb_to_intensity=False
)
# open3d.visualization.draw_geometries([o3d_rgbd])

# point cloud
pcd_gt = open3d.geometry.PointCloud.create_from_rgbd_image(
    image=o3d_rgbd,
    intrinsic=open3d.camera.PinholeCameraIntrinsic(width=width, height=height, intrinsic_matrix=intrinsics_gt),
    extrinsic=T,
    project_valid_depth_only=True
)

# open3d.visualization.draw_geometries([pcd_gt])


In [ ]:
# visualize input
fig, axs = plt.subplot_mosaic(
    [['RGB', 'Depth_GT']], figsize=(11, 11), layout='constrained')

plt_img1 = axs['RGB'].imshow(rgb)
axs['RGB'].set_axis_off()
axs['RGB'].set_title('RGB Input')

plt_img2 = axs['Depth_GT'].imshow(depth_gt, vmin=0.01, vmax=10, cmap='magma_r')
axs['Depth_GT'].set_axis_off()
axs['Depth_GT'].set_title('Depth GT')

plt.show()


In [ ]:
model = torch.hub.load("lpiccinelli-eth/UniDepth", "UniDepth",
                       version=version, backbone=backbone,
                       pretrained=True, trust_repo=True,
                       force_reload=True)
model = model.to(device)


In [ ]:
# predict
rgb_torch = torch.from_numpy(rgb).permute(2, 0, 1)
intrinsics_torch = torch.from_numpy(depth_intrinsics)
predictions = model.infer(rgb_torch, intrinsics_torch)
depth_pred = predictions["depth"].squeeze().cpu().numpy()
intrinsics_pred = predictions["intrinsics"].squeeze().cpu().numpy()
xyz_pred = predictions["points"]

# compute error, you have zero divison where depth_gt == 0.0
depth_arel = np.abs(depth_gt - depth_pred) / depth_gt
depth_arel[depth_gt == 0.0] = 0.0

print("Available predictions:", list(predictions.keys()))
print(f"ARel: {depth_arel[depth_gt > 0].mean() * 100:.2f}%")


In [ ]:
# visualize prediction
fig, axs = plt.subplot_mosaic(
    [['Depth_GT', 'Depth_Pred']], figsize=(11, 11), layout='constrained')

plt_img1 = axs['Depth_GT'].imshow(depth_gt, vmin=0, vmax=10, cmap='jet')
axs['Depth_GT'].set_axis_off()
axs['Depth_GT'].set_title('Depth GT')

plt_img2 = axs['Depth_Pred'].imshow(depth_pred, vmin=0, vmax=10, cmap='jet')
axs['Depth_Pred'].set_axis_off()
axs['Depth_Pred'].set_title('Depth Pred')

# save prediction
depth_pred_path = os.path.join(root, 'assets/3RScan/5630cfcf-12bf-2860-8784-83d28a611a83', 'prediction', f'frame-{frame_id:06}.depth.png')
cv2.imwrite(depth_pred_path, (depth_pred / integer_depth_scale).astype(np.uint16))

# visualize prediction
depth_pred_render_path = os.path.join(root, 'assets/3RScan/5630cfcf-12bf-2860-8784-83d28a611a83', 'rendered', f'frame-{frame_id:06}.depth.rendered.png')
depth_pred_col = cv2.rotate(depth_pred, cv2.ROTATE_90_CLOCKWISE)
depth_pred_col = colorize(depth_pred_col, vmin=0.01, vmax=10.0, cmap="magma_r")
Image.fromarray(depth_pred_col).save(depth_pred_render_path)

plt.show()


In [ ]:
# Open3D rgbd image
o3d_color = open3d.geometry.Image(rgb)
o3d_depth = open3d.geometry.Image(depth_pred)
o3d_rgbd = open3d.geometry.RGBDImage.create_from_color_and_depth(
    color=o3d_color,
    depth=o3d_depth,
    depth_scale=1.0,
    depth_trunc=10.0,
    convert_rgb_to_intensity=False
)
# open3d.visualization.draw_geometries([o3d_rgbd])

# point cloud
pcd_pred = open3d.geometry.PointCloud.create_from_rgbd_image(
    image=o3d_rgbd,
    intrinsic=open3d.camera.PinholeCameraIntrinsic(width=width, height=height, intrinsic_matrix=intrinsics_pred),
    extrinsic=T,
    project_valid_depth_only=True
)
# open3d.visualization.draw_geometries([pcd_pred])

# difference
np_colors = np.asarray(pcd_pred.colors)
np_colors[:] = np.array([1., 0., 0.])
pcd_pred.colors = open3d.utility.Vector3dVector(np_colors)

np_colors = np.asarray(pcd_gt.colors)
np_colors[:] = np.array([0., 1., 0.])
pcd_gt.colors = open3d.utility.Vector3dVector(np_colors)

# open3d.visualization.draw_geometries([pcd_pred, pcd_gt])
